## 9단계: 함수를 더 편리하게

> DeZero가 역전파와 Define-by-Run의 능력을 갖췄지만, 사용하기엔 여전히 불편한 감이 있습니다. \
이번 단계에서는 DeZero의 함수에 세 가지 개선을 추가하겠습니다.

### 9.1 파이썬 함수로 이용하기

지금까지 함수를 이용하려면 '인스턴스 생성', '호출' 단계들을 거쳤다.

하지만 함수를 먼저 초기화한 뒤에 다시 호출하는 형태는 단계도 늘어나고 보기도 좋지 않다.

먼저 이를 고쳐보자.

In [1]:
# 필요 모듈 정의

import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [2]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)


x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [3]:
# 연속하여 사용할 수도 있다.
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### 9.2 backward 메서드 간소화

`y.grad = np.array(1.0)`의 생략

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        # grad가 None일 때 자동으로 미분값 생성
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


x = Variable(np.array(0.5))
y = square(exp(square(x)))
# y.grad = np.array(1.0)  # 제거
y.backward()
print(x.grad)

3.297442541400256


### 9.3 ndarray만 취급하기

Variable이 None과 ndarray만 취급하도록 강제

In [18]:
class Variable:
    def __init__(self, data):
        # 밑시딥 원본 코드
        # if data is not None:
        #     if not isinstance(data, np.ndarray):
        #         raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        # 간소화한 코드
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        # grad가 None일 때 자동으로 미분값 생성성
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


x = Variable(np.array(1.0))  # OK
x = Variable(None)  # OK
x = Variable(1.0)  # Error

TypeError: <class 'float'>은(는) 지원하지 않습니다.

이로 인해 Function에서 output을 생성할 때 주의할 점이 생긴다.

numpy는 연산 결과가 scalar(0차원)이면 자동으로 np.float64, np.int64 등으로 결과를 반환한다.

이를 위해 Function에서 연산 결과가 scalar이면 ndarray로 바꿔주는 연산을 추가한다.

In [ ]:
# numpy 연산 결과 확인
x = np.array(1.0)
y = x ** 2  # 연산 결과가 scalar(0차원)

print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [21]:
# scalar이면 ndarray로 변환해주는 함수 추가
def as_array(x):
    if np.isscalar(x):
        # return np.array(x)  # 사실 이 코드는 메모리 복사를 일으킨다.
        return np.asarray(x)  # 수정: 메모리 복사 없이 ndarray 반환
    return x


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))  # 추가: ndarray 강제
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

Q. 왜 Variable 단에서 scalar를 처리 안 하고 Function에서 할까?

추후 답변 예정